In [137]:

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import os

In [138]:
# import geopandas as gpd
# import matplotlib.pyplot as plt

# # Define the paths
# input_file_path = "./antarctic_index.gpkg"
# map_path = "/Users/nathanbekele/Downloads/Quantarctica3/Miscellaneous/SimpleBasemap/ADD_DerivedLowresBasemap.shp"

# # Read the GeoPackage file
# gdf = gpd.read_file(input_file_path)

# # Print the first few rows of the GeoDataFrame
# print("First few rows of the GeoDataFrame:")
# print(gdf.head())

# # Display unique institutions to check if 'AWI' is present
# print("\nUnique institutions in the dataset:")
# print(gdf['institution'].unique())

# # Filter the GeoDataFrame for the institution 'AWI'
# filtered_gdf = gdf[gdf['institution'] == 'AWI']

# # Print the unique institutions in the filtered GeoDataFrame
# print("\nUnique institutions in the filtered GeoDataFrame:")
# print(filtered_gdf['institution'].unique())

# # Check if the filtered GeoDataFrame is empty
# print(f"\nFiltered GeoDataFrame for AWI is empty: {filtered_gdf.empty}")

# # Print the first few rows of the filtered GeoDataFrame to confirm filtering
# print("\nFirst few rows of the filtered GeoDataFrame:")
# print(filtered_gdf.head())

# # Print the CRS of the filtered GeoDataFrame
# print("\nCRS of the filtered GeoDataFrame before transformation:")
# print(filtered_gdf.crs)

# # Read the basemap shapefile
# basemap = gpd.read_file(map_path)

# # Print the CRS of the basemap
# print("\nCRS of the basemap:")
# print(basemap.crs)

# # Ensure the CRS matches between the basemap and the GeoDataFrame
# if not filtered_gdf.empty:
#     filtered_gdf = filtered_gdf.to_crs(basemap.crs)

# # Print the CRS of the filtered GeoDataFrame after transformation
# print("\nCRS of the filtered GeoDataFrame after transformation:")
# print(filtered_gdf.crs)

# # Function to plot data on the basemap
# def plot_data_on_basemap(basemap, gdf):
#     fig, ax = plt.subplots(figsize=(10, 10))

#     # Set background color to light blue
#     fig.patch.set_facecolor('lightblue')
#     ax.set_facecolor('lightblue')
    
#     # Plot basemap with Antarctica as white
#     basemap.plot(ax=ax, color='white', edgecolor='black')
    
#     # Plot the data
#     if not gdf.empty:
#         print("\nPlotting data:")
#         print(gdf)
#         gdf.plot(ax=ax, color='red', linewidth=2)
#     else:
#         print("GeoDataFrame is empty, nothing to plot.")

#     # Set limits to zoom in on Antarctica
#     ax.set_xlim(-3e6, 3e6)
#     ax.set_ylim(-3e6, 3e6)
    
#     # Remove axis for cleaner look
#     ax.set_axis_off()
    
#     plt.title('Antarctica Basemap with AWI Data Overlay')
#     plt.show()

# # Plot the data
# plot_data_on_basemap(basemap, filtered_gdf)


In [139]:
print(basemap.columns)

Index(['Category', 'geometry'], dtype='object')


In [140]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import os

# Define the paths
input_file_path = "./antarctic_index.gpkg"
map_path = "/Users/nathanbekele/Downloads/Quantarctica3/Miscellaneous/SimpleBasemap/ADD_DerivedLowresBasemap.shp"
output_folder = "./data"

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# List all layers in the GeoPackage
layers = gpd.io.file.fiona.listlayers(input_file_path)
print("Layers in the GeoPackage:")
print(layers)

# Create a dictionary to map institutions to layers
institution_layers = {}

# Populate the dictionary
for layer in layers:
    gdf_head = gpd.read_file(input_file_path, layer=layer, rows=10)  # Read first 10 rows
    gdf_tail = gpd.read_file(input_file_path, layer=layer, skiprows=lambda x: x < (gdf_head.shape[0] - 10))  # Read last 10 rows
    if 'institution' in gdf_head.columns:
        institutions = set(gdf_head['institution'].unique()).union(set(gdf_tail['institution'].unique()))
        for institution in institutions:
            if institution is not None:
                if institution not in institution_layers:
                    institution_layers[institution] = []
                institution_layers[institution].append(layer)

print("Institution layers mapping:")
print(institution_layers)

# Read the basemap shapefile
basemap = gpd.read_file(map_path)

# Function to plot data on the basemap with color-coding based on availability
def plot_data_on_basemap(basemap, gdf, institution):
    fig, ax = plt.subplots(figsize=(10, 10))

    # Set background color to light blue
    fig.patch.set_facecolor('lightblue')
    ax.set_facecolor('lightblue')
    
    # Plot basemap with Antarctica as white
    basemap.plot(ax=ax, color='white', edgecolor='black')
    
    # Define colors based on availability
    colors = {'u': 'darkblue', 's': 'red'}
    
    # Plot the data for the institution
    if not gdf.empty:
        for availability in gdf['availability'].unique():
            subset = gdf[gdf['availability'] == availability]
            color = colors.get(availability, 'black')
            subset.plot(ax=ax, color=color, markersize=1, linewidth=0.5, label=f"Availability {availability}")
    else:
        print("GeoDataFrame is empty, nothing to plot.")

    # Set limits to zoom in on Antarctica
    ax.set_xlim(-3e6, 3e6)
    ax.set_ylim(-3e6, 3e6)
    
    # Remove axis for cleaner look
    ax.set_axis_off()
    
    plt.title(f'Antarctica Basemap with {institution} Data Overlay')
    plt.legend()
    
    # Save the plot to the data folder
    output_path = os.path.join(output_folder, f'Antarctica_coverage_{institution}.png')
    plt.savefig(output_path)
    plt.close(fig)
    print(f"Saved map for {institution} to {output_path}")

# Iterate through each institution and create maps
for institution, layers in institution_layers.items():
    institution_data = []

    print(f"\nProcessing institution: {institution}")
    
    for layer in layers:
        print(f"Checking layer: {layer}")
        gdf = gpd.read_file(input_file_path, layer=layer)  # Read the full layer
        
        if 'institution' in gdf.columns and institution in gdf['institution'].unique():
            print(f"Layer '{layer}' contains '{institution}'.")
            institution_data.append(gdf[gdf['institution'] == institution])
        else:
            print(f"Layer '{layer}' does not contain '{institution}' or does not have 'institution' column.")

    # Combine all dataframes for the institution
    if institution_data:
        institution_gdf = gpd.GeoDataFrame(pd.concat(institution_data, ignore_index=True))
    else:
        institution_gdf = gpd.GeoDataFrame()

    # Ensure the CRS matches between the basemap and the GeoDataFrame
    if not institution_gdf.empty:
        institution_gdf = institution_gdf.to_crs(basemap.crs)

    # Plot and save the aggregated data for the institution
    plot_data_on_basemap(basemap, institution_gdf, institution)

print("All maps have been created and saved to the data folder.")


Layers in the GeoPackage:
['AWI_1994_DML1', 'AWI_1995_DML2', 'AWI_1996_DML3', 'AWI_1997_DML4', 'AWI_1998_DML5', 'AWI_2000_DML6', 'AWI_2001_DML7', 'AWI_2002_DML8', 'AWI_2003_DML9', 'AWI_2004_DML10', 'AWI_2005_ANTSYSO', 'AWI_2007_ANTR', 'AWI_2013_GEA-IV', 'AWI_2014_Recovery-Glacier', 'AWI_2015_GEA-DML', 'AWI_2016_OIR', 'AWI_2018_ANIRES', 'AWI_2018_DML-Coast', 'AWI_2018_JURAS', 'AWI_2019_JURAS', 'BAS_1994_Evans', 'BAS_1998_Dufek', 'BAS_2001_Bailey-Slessor', 'BAS_2001_MAMOG', 'BAS_2002_TORUS', 'BAS_2007_Lake-Ellsworth', 'BAS_2007_Rutford', 'BAS_2007_TIGRIS', 'BAS_2008_Lake-Ellsworth', 'BAS_2009_FERRIGNO', 'BAS_2010_PIG', 'BAS_2011_Adelaide', 'BAS_2012_Castle', 'BAS_2013_ISTAR', 'BAS_2018_Thwaites', 'BAS_2019_Thwaites', 'BEDMAP1_BEDMAP1', 'BGR_1999_GANOVEX-VIII-Matusevich', 'BGR_1999_GANOVEX-VIII-Mertz', 'BGR_2002_PCMEGA', 'CECS_2006_Subglacial-Lake-CECs', 'INGV_1997_ITASE', 'INGV_1997_Talos-Dome', 'INGV_1999_Talos-Dome', 'INGV_2001_Talos-Dome', 'INGV_2003_Talos-Dome', 'KOPRI_2017_KRT1', 'K

/var/folders/vv/czt16t1x4_12h0xsl6rcd5_80000gn/T/ipykernel_81961/2755912135.py:59: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  subset.plot(ax=ax, color=color, markersize=1, linewidth=0.5, label=f"Availability {availability}")


Saved map for BAS to ./data/Antarctica_coverage_BAS.png

Processing institution: BEDMAP1
Checking layer: BEDMAP1_BEDMAP1
Layer 'BEDMAP1_BEDMAP1' contains 'BEDMAP1'.
Saved map for BEDMAP1 to ./data/Antarctica_coverage_BEDMAP1.png

Processing institution: BGR
Checking layer: BGR_1999_GANOVEX-VIII-Matusevich
Layer 'BGR_1999_GANOVEX-VIII-Matusevich' contains 'BGR'.
Checking layer: BGR_1999_GANOVEX-VIII-Mertz
Layer 'BGR_1999_GANOVEX-VIII-Mertz' contains 'BGR'.
Checking layer: BGR_2002_PCMEGA
Layer 'BGR_2002_PCMEGA' contains 'BGR'.
Saved map for BGR to ./data/Antarctica_coverage_BGR.png

Processing institution: CECS
Checking layer: CECS_2006_Subglacial-Lake-CECs
Layer 'CECS_2006_Subglacial-Lake-CECs' contains 'CECS'.
Saved map for CECS to ./data/Antarctica_coverage_CECS.png

Processing institution: INGV
Checking layer: INGV_1997_ITASE
Layer 'INGV_1997_ITASE' contains 'INGV'.
Checking layer: INGV_1997_Talos-Dome
Layer 'INGV_1997_Talos-Dome' contains 'INGV'.
Checking layer: INGV_1999_Talos-Dom